# Practical Exercises on Graph Algorithms

This notebook contains practical exercises for applying BFS, DFS, Dijkstra, and A* algorithms on the Paris road network.

**Prerequisites**: Complete the `BFS_DFS_Shortest_Path.ipynb` notebook to understand the algorithms.


In [ ]:
import osmnx as ox
import networkx as nx
import plotly.graph_objects as go
import numpy as np
from collections import deque
import time
from typing import Dict, List, Tuple, Optional, Set

print("Libraries imported successfully!")
print(f"NetworkX version: {nx.__version__}")
print(f"OSMnx version: {ox.__version__}")


## Utility Classes and Functions

To keep the notebook clean and maintainable, we'll organize code into reusable utility classes and functions.


In [ ]:
class GraphPathFinder:
    """Utility class for path finding algorithms"""
    
    def __init__(self, G: nx.Graph, weight: Optional[str] = None):
        self.G = G
        self.weight = weight
    
    def get_node_coords(self, node) -> Tuple[float, float]:
        """Get coordinates (lon, lat) of a node"""
        if 'x' in self.G.nodes[node] and 'y' in self.G.nodes[node]:
            return (self.G.nodes[node]['x'], self.G.nodes[node]['y'])
        elif 'lon' in self.G.nodes[node] and 'lat' in self.G.nodes[node]:
            return (self.G.nodes[node]['lon'], self.G.nodes[node]['lat'])
        return None, None
    
    def heuristic(self, u, v) -> float:
        """Heuristic for A*: Euclidean distance between two nodes"""
        try:
            coords_u = self.get_node_coords(u)
            coords_v = self.get_node_coords(v)
            if coords_u[0] is None or coords_v[0] is None:
                return 0
            return np.sqrt((coords_v[0] - coords_u[0])**2 + (coords_v[1] - coords_u[1])**2)
        except:
            return 0
    
    def find_closest_node(self, lon: float, lat: float) -> Tuple[Optional[int], float]:
        """Find the closest node to given coordinates"""
        min_dist = float('inf')
        closest_node = None
        
        for node in self.G.nodes():
            node_lon, node_lat = self.get_node_coords(node)
            if node_lon is not None and node_lat is not None:
                dist = np.sqrt((node_lon - lon)**2 + (node_lat - lat)**2)
                if dist < min_dist:
                    min_dist = dist
                    closest_node = node
        
        return closest_node, min_dist * 111000  # Convert to approximate meters
    
    def dfs_iterative(self, start: int, end: Optional[int] = None) -> Tuple[Set, Optional[List], List]:
        """Iterative DFS using a stack"""
        visited = set()
        stack = [start]
        order = []
        parent = {start: None}
        
        while stack:
            node = stack.pop()
            
            if node not in visited:
                visited.add(node)
                order.append(node)
                
                if end is not None and node == end:
                    path = []
                    current = end
                    while current is not None:
                        path.append(current)
                        current = parent[current]
                    return visited, list(reversed(path)), order
                
                neighbors = list(self.G.neighbors(node))
                neighbors.reverse()
                for neighbor in neighbors:
                    if neighbor not in visited:
                        stack.append(neighbor)
                        if neighbor not in parent:
                            parent[neighbor] = node
        
        if end is not None:
            return visited, None, order
        
        return visited, None, order
    
    def bfs(self, start: int, end: int) -> Optional[List]:
        """BFS implementation"""
        visited = set()
        queue = deque([start])
        visited.add(start)
        parent = {start: None}
        
        while queue:
            node = queue.popleft()
            
            if node == end:
                path = []
                current = end
                while current is not None:
                    path.append(current)
                    current = parent[current]
                return list(reversed(path))
            
            for neighbor in self.G.neighbors(node):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)
                    parent[neighbor] = node
        
        return None
    
    def dijkstra(self, start: int, end: int) -> Tuple[Optional[List], Optional[float]]:
        """Dijkstra's algorithm"""
        try:
            if self.weight:
                path = nx.shortest_path(self.G, source=start, target=end, weight=self.weight, method='dijkstra')
                length = nx.shortest_path_length(self.G, source=start, target=end, weight=self.weight)
                return path, length
            else:
                path = nx.shortest_path(self.G, source=start, target=end)
                return path, len(path)
        except nx.NetworkXNoPath:
            return None, None
        except Exception as e:
            print(f"Error in Dijkstra: {e}")
            return None, None
    
    def astar(self, start: int, end: int) -> Tuple[Optional[List], Optional[float]]:
        """A* algorithm"""
        try:
            h = lambda u, v: self.heuristic(u, v)
            if self.weight:
                path = nx.astar_path(self.G, source=start, target=end, weight=self.weight, heuristic=h)
                length = nx.astar_path_length(self.G, source=start, target=end, weight=self.weight, heuristic=h)
                return path, length
            else:
                path = nx.astar_path(self.G, source=start, target=end, heuristic=h)
                return path, len(path)
        except nx.NetworkXNoPath:
            return None, None
        except Exception as e:
            print(f"Error in A*: {e}")
            return None, None


In [ ]:
class GraphVisualizer:
    """Utility class for creating Plotly visualizations"""
    
    def __init__(self, G: nx.Graph):
        self.G = G
    
    def get_node_coords(self, node) -> Tuple[Optional[float], Optional[float]]:
        """Get coordinates (lon, lat) of a node"""
        if 'x' in self.G.nodes[node] and 'y' in self.G.nodes[node]:
            return (self.G.nodes[node]['x'], self.G.nodes[node]['y'])
        elif 'lon' in self.G.nodes[node] and 'lat' in self.G.nodes[node]:
            return (self.G.nodes[node]['lon'], self.G.nodes[node]['lat'])
        return None, None
    
    def get_all_coords(self, nodes: List) -> Dict:
        """Get coordinates for multiple nodes"""
        pos = {}
        for node in nodes:
            lon, lat = self.get_node_coords(node)
            if lon is not None and lat is not None:
                pos[node] = (lon, lat)
        return pos
    
    def create_subgraph(self, nodes: Set, max_nodes: int = 1000) -> nx.Graph:
        """Create a subgraph with nodes and their neighbors"""
        nodes_list = list(nodes)[:max_nodes]
        return self.G.subgraph(nodes_list).copy()
    
    def calculate_center(self, pos: Dict) -> Tuple[float, float]:
        """Calculate center coordinates from positions"""
        if pos:
            lons = [p[0] for p in pos.values() if isinstance(p[0], (int, float))]
            lats = [p[1] for p in pos.values() if isinstance(p[1], (int, float))]
            if lons and lats:
                return sum(lons) / len(lons), sum(lats) / len(lats)
        return 2.3522, 48.8566  # Default: Paris center
    
    def plot_paths(self, paths: Dict[str, List], 
                   highlight_nodes: Optional[List] = None,
                   highlight_points: Optional[Dict[str, Tuple[float, float]]] = None,
                   title: str = "Graph Visualization",
                   max_subgraph_nodes: int = 800) -> go.Figure:
        """Create a Plotly visualization with paths"""
        # Collect all path nodes
        all_path_nodes = set()
        for path in paths.values():
            if path:
                all_path_nodes.update(path)
                # Add some neighbors for context
                for node in path[::10]:
                    for neighbor in list(self.G.neighbors(node))[:2]:
                        all_path_nodes.add(neighbor)
        
        if not all_path_nodes:
            return None
        
        # Create subgraph
        G_sub = self.create_subgraph(all_path_nodes, max_subgraph_nodes)
        pos = self.get_all_coords(G_sub.nodes())
        
        if not pos:
            return None
        
        center_lon, center_lat = self.calculate_center(pos)
        
        fig = go.Figure()
        
        # Add edges (light gray)
        edge_lons, edge_lats = [], []
        for edge in G_sub.edges():
            if edge[0] in pos and edge[1] in pos:
                lon0, lat0 = pos[edge[0]]
                lon1, lat1 = pos[edge[1]]
                if (isinstance(lon0, (int, float)) and isinstance(lat0, (int, float)) and
                    isinstance(lon1, (int, float)) and isinstance(lat1, (int, float))):
                    edge_lons.extend([lon0, lon1, None])
                    edge_lats.extend([lat0, lat1, None])
        
        if edge_lons:
            fig.add_trace(go.Scattermap(
                mode='lines',
                lon=edge_lons,
                lat=edge_lats,
                line=dict(width=0.3, color='lightgray'),
                hoverinfo='none',
                showlegend=False
            ))
        
        # Add paths with different colors
        colors = ['blue', 'green', 'purple', 'orange', 'red', 'cyan', 'magenta']
        for i, (name, path) in enumerate(paths.items()):
            if path:
                path_lons = [pos[node][0] for node in path if node in pos 
                            and isinstance(pos[node][0], (int, float))]
                path_lats = [pos[node][1] for node in path if node in pos 
                            and isinstance(pos[node][1], (int, float))]
                
                if path_lons and path_lats:
                    color = colors[i % len(colors)]
                    fig.add_trace(go.Scattermap(
                        mode='lines+markers',
                        lon=path_lons,
                        lat=path_lats,
                        line=dict(width=4, color=color),
                        marker=dict(size=5, color=color, opacity=0.6),
                        name=name,
                        showlegend=True
                    ))
        
        # Add highlight nodes
        if highlight_nodes:
            highlight_lons = [pos[node][0] for node in highlight_nodes if node in pos]
            highlight_lats = [pos[node][1] for node in highlight_nodes if node in pos]
            if highlight_lons:
                fig.add_trace(go.Scattermap(
                    mode='markers',
                    lon=highlight_lons,
                    lat=highlight_lats,
                    marker=dict(size=12, color='orange', opacity=0.8),
                    name='Highlighted nodes',
                    showlegend=True
                ))
        
        # Add highlight points (e.g., monuments, stations)
        if highlight_points:
            point_lons = [coords[0] for coords in highlight_points.values()]
            point_lats = [coords[1] for coords in highlight_points.values()]
            point_labels = list(highlight_points.keys())
            
            if point_lons:
                fig.add_trace(go.Scattermap(
                    mode='markers',
                    lon=point_lons,
                    lat=point_lats,
                    text=point_labels,
                    marker=dict(size=15, color='red', symbol='circle'),
                    name='Points of interest',
                    showlegend=True
                ))
        
        # Update layout
        fig.update_layout(
            title=title,
            showlegend=True,
            mapbox=dict(
                style="open-street-map",
                center=dict(lon=center_lon, lat=center_lat),
                zoom=12,
                bearing=0,
                pitch=0
            ),
            margin=dict(l=0, r=0, t=40, b=0),
            height=700
        )
        
        return fig
    
    def plot_network_analysis(self, nodes: List, node_sizes: List, 
                             node_labels: List, title: str = "Network Analysis") -> go.Figure:
        """Visualize network nodes with sizes and labels"""
        # Create subgraph with nodes and neighbors
        nodes_set = set(nodes)
        for node in nodes[:10]:
            nodes_set.update(list(self.G.neighbors(node))[:5])
        
        G_sub = self.create_subgraph(nodes_set, 200)
        pos = self.get_all_coords(G_sub.nodes())
        
        if not pos:
            return None
        
        center_lon, center_lat = self.calculate_center(pos)
        fig = go.Figure()
        
        # Add edges
        edge_lons, edge_lats = [], []
        for edge in G_sub.edges():
            if edge[0] in pos and edge[1] in pos:
                lon0, lat0 = pos[edge[0]]
                lon1, lat1 = pos[edge[1]]
                edge_lons.extend([lon0, lon1, None])
                edge_lats.extend([lat0, lat1, None])
        
        if edge_lons:
            fig.add_trace(go.Scattermap(
                mode='lines',
                lon=edge_lons,
                lat=edge_lats,
                line=dict(width=0.5, color='lightgray'),
                hoverinfo='none',
                showlegend=False
            ))
        
        # Add highlighted nodes
        highlight_lons = [pos[node][0] for node in nodes if node in pos]
        highlight_lats = [pos[node][1] for node in nodes if node in pos]
        highlight_labels = [label for node, label in zip(nodes, node_labels) if node in pos]
        highlight_sizes = [size for node, size in zip(nodes, node_sizes) if node in pos]
        
        if highlight_lons:
            fig.add_trace(go.Scattermap(
                mode='markers',
                lon=highlight_lons,
                lat=highlight_lats,
                text=highlight_labels,
                hoverinfo='text',
                marker=dict(
                    size=highlight_sizes,
                    color='red',
                    opacity=0.7
                ),
                name='Highlighted nodes',
                showlegend=True
            ))
        
        # Add other nodes
        other_nodes = [n for n in G_sub.nodes() if n not in nodes]
        other_lons = [pos[node][0] for node in other_nodes if node in pos]
        other_lats = [pos[node][1] for node in other_nodes if node in pos]
        
        if other_lons:
            fig.add_trace(go.Scattermap(
                mode='markers',
                lon=other_lons,
                lat=other_lats,
                marker=dict(size=4, color='lightblue', opacity=0.5),
                name='Other nodes',
                showlegend=True
            ))
        
        fig.update_layout(
            title=title,
            showlegend=True,
            mapbox=dict(
                style="open-street-map",
                center=dict(lon=center_lon, lat=center_lat),
                zoom=12,
                bearing=0,
                pitch=0
            ),
            margin=dict(l=0, r=0, t=40, b=0),
            height=700
        )
        
        return fig


In [ ]:
class NetworkAnalyzer:
    """Utility class for network analysis"""
    
    def __init__(self, G: nx.Graph, weight: Optional[str] = None):
        self.G = G
        self.weight = weight
    
    def get_basic_stats(self) -> Dict:
        """Get basic graph statistics"""
        return {
            'num_nodes': len(self.G.nodes()),
            'num_edges': len(self.G.edges()),
            'density': nx.density(self.G),
            'is_connected': nx.is_connected(self.G)
        }
    
    def analyze_degrees(self) -> Dict:
        """Analyze node degrees"""
        degrees = dict(self.G.degree())
        avg_degree = sum(degrees.values()) / len(degrees) if degrees else 0
        max_degree_node = max(degrees.items(), key=lambda x: x[1])
        min_degree_node = min(degrees.items(), key=lambda x: x[1])
        
        return {
            'degrees': degrees,
            'avg_degree': avg_degree,
            'max_degree': max_degree_node,
            'min_degree': min_degree_node
        }
    
    def analyze_path_lengths(self, sample_size: int = 50) -> Dict:
        """Analyze path lengths on a sample of node pairs"""
        sample_nodes = list(self.G.nodes())[:sample_size]
        path_lengths = []
        distances = []
        
        for i in range(min(50, sample_size - 1)):
            try:
                if self.weight:
                    length = nx.shortest_path_length(
                        self.G, 
                        source=sample_nodes[i], 
                        target=sample_nodes[i+1], 
                        weight=self.weight
                    )
                    distances.append(length)
                else:
                    path = nx.shortest_path(
                        self.G, 
                        source=sample_nodes[i], 
                        target=sample_nodes[i+1]
                    )
                    path_lengths.append(len(path))
            except (nx.NetworkXNoPath, IndexError):
                continue
        
        result = {}
        if distances:
            result = {
                'avg_distance': sum(distances) / len(distances),
                'max_distance': max(distances),
                'min_distance': min(distances),
                'distances': distances
            }
        elif path_lengths:
            result = {
                'avg_path_length': sum(path_lengths) / len(path_lengths),
                'path_lengths': path_lengths
            }
        
        return result
    
    def get_top_connected_nodes(self, n: int = 5) -> List[Tuple]:
        """Get top n most connected nodes"""
        degrees = dict(self.G.degree())
        sorted_nodes = sorted(degrees.items(), key=lambda x: x[1], reverse=True)[:n]
        return sorted_nodes


## Loading the Paris Road Network

We'll load the road network graph of Paris using OSMnx.


In [ ]:
# Load the Paris road network graph
print("Loading Paris road network...")
G = ox.graph_from_place("Paris, France", network_type="drive")

# Convert to undirected graph to simplify algorithms
G = G.to_undirected()

print(f"Graph loaded: {len(G.nodes())} nodes, {len(G.edges())} edges")
print(f"Graph type: {type(G)}")
print(f"Is connected: {nx.is_connected(G)}")

# Select largest connected component if needed
if not nx.is_connected(G):
    print("Graph is not connected. Selecting largest connected component...")
    largest_cc = max(nx.connected_components(G), key=len)
    G = G.subgraph(largest_cc).copy()
    print(f"New graph: {len(G.nodes())} nodes, {len(G.edges())} edges")

# Determine weight attribute
weight = None
if len(G.edges()) > 0:
    sample_edge = list(G.edges(keys=True))[0] if isinstance(G, nx.MultiGraph) else list(G.edges())[0]
    
    if isinstance(G, nx.MultiGraph):
        edge_data = G[sample_edge[0]][sample_edge[1]][sample_edge[2]]
    else:
        edge_data = G[sample_edge[0]][sample_edge[1]]
    
    if 'length' in edge_data:
        weight = 'length'
        print(f"\n✓ Weight attribute detected: 'length' (distance in meters)")
    else:
        print(f"\n⚠ No 'length' weight attribute found. Weighted algorithms will be disabled.")
        print(f"  Available edge attributes: {list(edge_data.keys())[:5]}")

# Initialize utility classes
path_finder = GraphPathFinder(G, weight)
visualizer = GraphVisualizer(G)
analyzer = NetworkAnalyzer(G, weight)

# Select test nodes
nodes = list(G.nodes())
start_node = nodes[0]
end_node = nodes[len(nodes)//4]

print(f"\nStart node: {start_node}")
print(f"End node: {end_node}")


## Exercise 1: Finding Paths Between Parisian Monuments

This exercise demonstrates how to find nodes near famous Parisian monuments and visualize paths between them.


In [ ]:
# Exercise 1: Find paths between Parisian monuments
try:
    # Coordinates of famous monuments
    monuments = {
        'Eiffel Tower': (2.2945, 48.8584),
        'Notre-Dame': (2.3499, 48.8530),
        'Louvre': (2.3364, 48.8606),
        'Arc de Triomphe': (2.2950, 48.8738),
        'Sacré-Cœur': (2.3431, 48.8867),
        'Gare du Nord': (2.3553, 48.8809)
    }
    
    print("=" * 70)
    print("EXERCISE 1: Finding nodes near Parisian monuments")
    print("=" * 70)
    print("\nSearching for nodes near monuments...\n")
    
    monument_nodes = {}
    monument_coords = {}
    
    for name, (lon, lat) in monuments.items():
        closest_node, dist_m = path_finder.find_closest_node(lon, lat)
        if closest_node is not None:
            monument_nodes[name] = closest_node
            node_lon, node_lat = path_finder.get_node_coords(closest_node)
            monument_coords[name] = (node_lon, node_lat)
            print(f"{name:20s}: node {closest_node} (distance: {dist_m:.0f} m)")
    
    # Find paths between monument pairs
    print("\n" + "=" * 70)
    print("Paths between monuments (using Dijkstra)")
    print("=" * 70)
    
    pairs_to_test = [
        ('Eiffel Tower', 'Louvre'),
        ('Arc de Triomphe', 'Sacré-Cœur'),
        ('Notre-Dame', 'Gare du Nord')
    ]
    
    all_paths = {}
    for start_monument, end_monument in pairs_to_test:
        if start_monument in monument_nodes and end_monument in monument_nodes:
            start_m = monument_nodes[start_monument]
            end_m = monument_nodes[end_monument]
            
            path_monument, length_monument = path_finder.dijkstra(start_m, end_m)
            
            if path_monument:
                all_paths[(start_monument, end_monument)] = (path_monument, length_monument)
                
                print(f"\n{start_monument} → {end_monument}:")
                print(f"  - Number of nodes: {len(path_monument)}")
                if weight:
                    print(f"  - Total distance: {length_monument:.0f} meters")
                    print(f"  - Distance in km: {length_monument/1000:.2f} km")
                else:
                    print(f"  - Distance: {length_monument:.2f}")
    
    # Visualize paths
    if all_paths:
        print("\n" + "=" * 70)
        print("Visualizing paths between monuments")
        print("=" * 70)
        
        paths_dict = {f"{start} → {end}": path for (start, end), (path, _) in all_paths.items()}
        highlight_points = {name: coords for name, coords in monument_coords.items()}
        
        fig = visualizer.plot_paths(
            paths_dict,
            highlight_points=highlight_points,
            title="Paths Between Parisian Monuments"
        )
        
        if fig:
            fig.show()
        
except Exception as e:
    print(f"Error in Exercise 1: {e}")
    import traceback
    traceback.print_exc()


## Exercise 2: Analyzing the Paris Road Network

This exercise analyzes the characteristics of the Paris road network, including statistics, node degrees, and path lengths.


In [ ]:
# Exercise 2: Analyze the Paris road network
try:
    print("=" * 70)
    print("EXERCISE 2: Paris Road Network Analysis")
    print("=" * 70)
    
    # Basic statistics
    stats = analyzer.get_basic_stats()
    print("\n1. Graph Statistics:")
    print(f"   - Number of nodes: {stats['num_nodes']}")
    print(f"   - Number of edges: {stats['num_edges']}")
    print(f"   - Density: {stats['density']:.6f}")
    print(f"   - Is connected: {stats['is_connected']}")
    
    # Degree analysis
    degree_info = analyzer.analyze_degrees()
    print("\n2. Degree Analysis:")
    print(f"   - Average degree: {degree_info['avg_degree']:.2f}")
    print(f"   - Most connected node: {degree_info['max_degree'][0]} (degree {degree_info['max_degree'][1]})")
    print(f"   - Least connected node: {degree_info['min_degree'][0]} (degree {degree_info['min_degree'][1]})")
    
    # Path length analysis
    path_info = analyzer.analyze_path_lengths(50)
    print("\n3. Distance Analysis (sample of 50 pairs):")
    if 'avg_distance' in path_info:
        print(f"   - Average distance: {path_info['avg_distance']:.0f} meters ({path_info['avg_distance']/1000:.2f} km)")
        print(f"   - Maximum distance: {path_info['max_distance']:.0f} meters ({path_info['max_distance']/1000:.2f} km)")
        print(f"   - Minimum distance: {path_info['min_distance']:.0f} meters ({path_info['min_distance']/1000:.2f} km)")
    elif 'avg_path_length' in path_info:
        print(f"   - Average path length: {path_info['avg_path_length']:.1f} nodes")
    
    # Top connected nodes
    top_nodes = analyzer.get_top_connected_nodes(5)
    print("\n4. Top 5 Most Connected Nodes:")
    for node, deg in top_nodes:
        node_lon, node_lat = path_finder.get_node_coords(node)
        if node_lon is not None:
            print(f"   - Node {node}: degree {deg} (coords: {node_lon:.4f}, {node_lat:.4f})")
    
    print("\n✓ Analysis complete!")
    
    # Visualization: Show most connected nodes
    print("\n" + "=" * 70)
    print("Visualizing most connected nodes")
    print("=" * 70)
    
    top_nodes_list = analyzer.get_top_connected_nodes(15)
    top_node_ids = [node for node, _ in top_nodes_list]
    top_node_sizes = [min(20, 8 + deg) for _, deg in top_nodes_list]
    top_node_labels = [f"Node {node}<br>Degree: {deg}" for node, deg in top_nodes_list]
    
    fig = visualizer.plot_network_analysis(
        top_node_ids,
        top_node_sizes,
        top_node_labels,
        title="Most Connected Nodes in Paris Road Network"
    )
    
    if fig:
        fig.show()
        print("   ✓ Visualization created")
    
except Exception as e:
    print(f"Error in Exercise 2: {e}")
    import traceback
    traceback.print_exc()


## Exercise 3: Comparing BFS, DFS, Dijkstra, and A* Algorithms

This exercise compares the performance and results of different graph traversal algorithms on various routes.


In [ ]:
# Exercise 3: Compare BFS, DFS, Dijkstra, and A* on different routes
try:
    print("=" * 70)
    print("EXERCISE 3: Comparison of BFS, DFS, Dijkstra, and A*")
    print("=" * 70)
    
    # Select multiple node pairs for testing
    nodes_list = list(G.nodes())
    test_pairs = []
    
    if len(nodes_list) >= 100:
        test_pairs = [
            (nodes_list[0], nodes_list[len(nodes_list)//10]),      # Short distance
            (nodes_list[0], nodes_list[len(nodes_list)//4]),       # Medium distance
            (nodes_list[0], nodes_list[len(nodes_list)//2])        # Long distance
        ]
    else:
        test_pairs = [
            (nodes_list[0], nodes_list[min(10, len(nodes_list)-1)]),
            (nodes_list[0], nodes_list[min(20, len(nodes_list)-1)])
        ]
    
    print("\nComparison for multiple routes:\n")
    
    all_results = []
    
    for i, (start, end) in enumerate(test_pairs, 1):
        print(f"{'='*70}")
        print(f"Route {i}: Node {start} → Node {end}")
        print(f"{'='*70}")
        
        results = {}
        
        # BFS
        try:
            t_start = time.time()
            path_bfs = path_finder.bfs(start, end)
            t_bfs = time.time() - t_start
            if path_bfs:
                results['BFS'] = {
                    'path': path_bfs,
                    'length': len(path_bfs),
                    'time': t_bfs
                }
                print(f"BFS      : {len(path_bfs):4d} nodes, time: {t_bfs*1000:6.2f} ms")
            else:
                print(f"BFS      : No path found")
                results['BFS'] = None
        except Exception as e:
            print(f"BFS      : Error - {e}")
            results['BFS'] = None
        
        # DFS
        try:
            t_start = time.time()
            visited_dfs, path_dfs, _ = path_finder.dfs_iterative(start, end)
            t_dfs = time.time() - t_start
            if path_dfs:
                results['DFS'] = {
                    'path': path_dfs,
                    'length': len(path_dfs),
                    'time': t_dfs
                }
                print(f"DFS      : {len(path_dfs):4d} nodes, time: {t_dfs*1000:6.2f} ms")
            else:
                print(f"DFS      : No path found")
                results['DFS'] = None
        except Exception as e:
            print(f"DFS      : Error - {e}")
            results['DFS'] = None
        
        # Dijkstra
        if weight:
            try:
                t_start = time.time()
                path_dijk, dist_dijk = path_finder.dijkstra(start, end)
                t_dijk = time.time() - t_start
                if path_dijk:
                    results['Dijkstra'] = {
                        'path': path_dijk,
                        'length': len(path_dijk),
                        'distance': dist_dijk,
                        'time': t_dijk
                    }
                    print(f"Dijkstra : {len(path_dijk):4d} nodes, {dist_dijk:.0f} m, time: {t_dijk*1000:6.2f} ms")
                else:
                    print(f"Dijkstra : No path found")
                    results['Dijkstra'] = None
            except Exception as e:
                print(f"Dijkstra : Error - {e}")
                results['Dijkstra'] = None
        
        # A*
        if weight:
            try:
                t_start = time.time()
                path_ast, dist_ast = path_finder.astar(start, end)
                t_ast = time.time() - t_start
                if path_ast:
                    results['A*'] = {
                        'path': path_ast,
                        'length': len(path_ast),
                        'distance': dist_ast,
                        'time': t_ast
                    }
                    print(f"A*       : {len(path_ast):4d} nodes, {dist_ast:.0f} m, time: {t_ast*1000:6.2f} ms")
                else:
                    print(f"A*       : No path found")
                    results['A*'] = None
            except Exception as e:
                print(f"A*       : Error - {e}")
                results['A*'] = None
        
        all_results.append((start, end, results))
        print()
    
    # Summary
    print("\n" + "=" * 70)
    print("Summary of Results:")
    print("=" * 70)
    
    if weight:
        print("\nOn a weighted graph:")
        print("- BFS finds the shortest path in number of nodes (but not necessarily in distance)")
        print("- Dijkstra and A* find the shortest path in distance")
        print("- A* is generally faster than Dijkstra thanks to the heuristic")
    
    print("\nObservations:")
    print("- BFS guarantees the shortest path in number of nodes on an unweighted graph")
    print("- DFS does NOT guarantee the shortest path")
    print("- For real paths (with distances), use Dijkstra or A*")
    
except Exception as e:
    print(f"Error in Exercise 3: {e}")
    import traceback
    traceback.print_exc()


## Exercise 4: Finding Optimal Paths Between Parisian Train Stations

This exercise finds optimal paths between major Parisian train stations and creates a distance matrix.


In [ ]:
# Exercise 4: Find optimal paths between Parisian train stations
try:
    print("=" * 70)
    print("EXERCISE 4: Paths Between Parisian Train Stations")
    print("=" * 70)
    
    # Coordinates of major Parisian train stations
    stations = {
        'Gare du Nord': (2.3553, 48.8809),
        "Gare de l'Est": (2.3590, 48.8768),
        'Gare de Lyon': (2.3733, 48.8447),
        'Gare Montparnasse': (2.3217, 48.8412),
        'Gare Saint-Lazare': (2.3260, 48.8767),
        "Gare d'Austerlitz": (2.3642, 48.8420)
    }
    
    print("\nSearching for nodes near stations...\n")
    
    station_nodes = {}
    station_coords = {}
    for name, (lon, lat) in stations.items():
        closest_node, dist_m = path_finder.find_closest_node(lon, lat)
        if closest_node:
            station_nodes[name] = closest_node
            node_lon, node_lat = path_finder.get_node_coords(closest_node)
            station_coords[name] = (node_lon, node_lat)
            print(f"{name:25s}: node {closest_node} (distance: {dist_m:.0f} m)")
    
    # Calculate paths between all station pairs
    print("\n" + "=" * 70)
    print("Distances between stations (distance matrix)")
    print("=" * 70)
    
    station_names = list(station_nodes.keys())
    paths_matrix = {}
    distances_matrix = {}
    
    for start_station in station_names:
        for end_station in station_names:
            if start_station != end_station:
                if start_station in station_nodes and end_station in station_nodes:
                    start_node = station_nodes[start_station]
                    end_node = station_nodes[end_station]
                    
                    path, dist = path_finder.dijkstra(start_node, end_node)
                    if path:
                        paths_matrix[(start_station, end_station)] = path
                        if dist:
                            distances_matrix[(start_station, end_station)] = dist
    
    # Display distance matrix
    if distances_matrix:
        print("\nDistances in meters between stations:")
        print(f"\n{'Station':<20s}", end="")
        for station in station_names[:5]:
            print(f"{station[:15]:>15s}", end="")
        print()
        print("-" * 90)
        
        for start in station_names[:5]:
            print(f"{start:<20s}", end="")
            for end in station_names[:5]:
                if start == end:
                    print(f"{'0':>15s}", end="")
                elif (start, end) in distances_matrix:
                    dist = distances_matrix[(start, end)]
                    print(f"{dist/1000:>13.2f}km", end="")
                else:
                    print(f"{'N/A':>15s}", end="")
            print()
        
        # Find closest and farthest pairs
        if distances_matrix:
            min_pair = min(distances_matrix.items(), key=lambda x: x[1])
            max_pair = max(distances_matrix.items(), key=lambda x: x[1])
            
            print(f"\nClosest stations: {min_pair[0][0]} ↔ {min_pair[0][1]}")
            print(f"  Distance: {min_pair[1]:.0f} meters ({min_pair[1]/1000:.2f} km)")
            
            print(f"\nFarthest stations: {max_pair[0][0]} ↔ {max_pair[0][1]}")
            print(f"  Distance: {max_pair[1]:.0f} meters ({max_pair[1]/1000:.2f} km)")
    
    # Visualize paths between main stations
    if paths_matrix:
        print("\n" + "=" * 70)
        print("Visualizing paths between main stations")
        print("=" * 70)
        
        # Select interesting paths to visualize
        paths_to_show = []
        if ('Gare du Nord', 'Gare de Lyon') in paths_matrix:
            paths_to_show.append(('Gare du Nord', 'Gare de Lyon'))
        if ('Gare Montparnasse', 'Gare du Nord') in paths_matrix:
            paths_to_show.append(('Gare Montparnasse', 'Gare du Nord'))
        if ('Gare de Lyon', 'Gare Saint-Lazare') in paths_matrix:
            paths_to_show.append(('Gare de Lyon', 'Gare Saint-Lazare'))
        
        if paths_to_show:
            paths_dict = {
                f"{start} → {end}": paths_matrix[(start, end)]
                for start, end in paths_to_show
            }
            
            highlight_points = {name: coords for name, coords in station_coords.items()}
            
            fig = visualizer.plot_paths(
                paths_dict,
                highlight_points=highlight_points,
                title="Paths Between Parisian Train Stations"
            )
            
            if fig:
                fig.show()
    
except Exception as e:
    print(f"Error in Exercise 4: {e}")
    import traceback
    traceback.print_exc()
